In [3]:

import numpy as np

import rasterio
from rasterio.mask import mask
from shapely.geometry import box
from rasterio.warp import transform_bounds
from xml.etree import ElementTree

import geopandas as gpd

from osgeo import gdal


In [ ]:

# Path to B02 .jp2
jp2_path = r'S2A_MSIL2A_20181227T055231_N0500_R048_T43SCT_20230726T094312.SAFE\GRANULE\L2A_T43SCT_A018349_20181227T055233\IMG_DATA\R10m\T43SCT_20181227T055231_B02_10m.jp2'

# Inspect
with rasterio.open(jp2_path) as src:
    print("Band: B02")
    print("CRS:", src.crs)  # Should be EPSG:32643
    print("Bounds:", src.bounds)
    print("Shape:", src.shape)
    print("Bands:", src.count)
    bounds_wgs84 = transform_bounds(src.crs, 'EPSG:4326', *src.bounds)
    print("Bounds in WGS84 (degrees):", bounds_wgs84)
    

Band: B02
CRS: EPSG:32643
Bounds: BoundingBox(left=300000.0, bottom=3690240.0, right=409800.0, top=3800040.0)
Shape: (10980, 10980)
Bands: 1
Bounds in WGS84 (degrees): (72.82621993951749, 33.33276710714379, 74.03062458149928, 34.33773660758858)


In [28]:
xml_path = r"S2A_MSIL2A_20181227T055231_N0500_R048_T43SCT_20230726T094312.SAFE\MTD_MSIL2A.xml"
tree = ElementTree.parse(xml_path)
root = tree.getroot()

# Print all tags to debug
for elem in root.iter():
    print(elem.tag, elem.text)

{https://psd-14.sentinel2.eo.esa.int/PSD/User_Product_Level-2A.xsd}Level-2A_User_Product 
    
{https://psd-14.sentinel2.eo.esa.int/PSD/User_Product_Level-2A.xsd}General_Info 
        
Product_Info 
            
PRODUCT_START_TIME 2018-12-27T05:52:31.024Z
PRODUCT_STOP_TIME 2018-12-27T05:52:31.024Z
PRODUCT_URI S2A_MSIL2A_20181227T055231_N0500_R048_T43SCT_20230726T094312.SAFE
PROCESSING_LEVEL Level-2A
PRODUCT_TYPE S2MSI2A
PROCESSING_BASELINE 05.00
PRODUCT_DOI https://doi.org/10.5270/S2_-znk9xsj
GENERATION_TIME 2023-07-26T09:43:12.000000Z
PREVIEW_IMAGE_URL Not applicable
PREVIEW_GEO_INFO Not applicable
Datatake 
      
SPACECRAFT_NAME Sentinel-2A
DATATAKE_TYPE INS-NOBS
DATATAKE_SENSING_START 2018-12-27T05:52:31.024Z
SENSING_ORBIT_NUMBER 48
SENSING_ORBIT_DIRECTION DESCENDING
Query_Options 

PRODUCT_FORMAT SAFE_COMPACT
Product_Organisation 
                
Granule_List 
                    
Granule 
                        
IMAGE_FILE GRANULE/L2A_T43SCT_A018349_20181227T055233/IMG_DATA/R10

In [30]:
scl_path = r"S2A_MSIL2A_20181227T055231_N0500_R048_T43SCT_20230726T094312.SAFE\GRANULE\L2A_T43SCT_A018349_20181227T055233\IMG_DATA\R10m\T43SCT_20181227T055231_B02_10m.jp2"
with rasterio.open(scl_path) as src:
    scl = src.read(1)
    cloud_pixels = np.isin(scl, [8, 9, 10]).sum()  # Clouds: low, high, cirrus
    total_pixels = scl.size
    cloud_percentage = (cloud_pixels / total_pixels) * 100
    print("Estimated Cloud Cover (%):", cloud_percentage)

Estimated Cloud Cover (%): 0.0


In [31]:

# GeoTIFF bounds in ESRI:54009
bounds_mollweide = (5959000.0, 4000000.0, 6959000.0, 5000000.0)  # left, bottom, right, top

# Convert to EPSG:4326
bounds_wgs84 = transform_bounds('ESRI:54009', 'EPSG:4326', *bounds_mollweide)
print("Bounds in WGS84 (degrees):", bounds_wgs84)  # [min_lon, min_lat, max_lon, max_lat]

Bounds in WGS84 (degrees): (66.33697696385329, 33.06103686093294, 83.42553240338171, 41.894152150793836)


In [32]:
bbox = [73.0, 33.6, 73.2, 33.8]
geom = box(*bbox)
gdf = gpd.GeoDataFrame({'geometry': [geom]}, crs='EPSG:4326')

In [16]:

gdf = gdf.to_crs('ESRI:54009')

tif_path = r'D:\Ass\ANN\Project\GHS BUILT C\GHS_BUILT_C_FUN_E2018_GLOBE_R2023A_54009_10_V1_0_R5_C25.tif'  
with rasterio.open(tif_path) as src:
    try:
        clipped, transform = mask(src, gdf.geometry, crop=True, all_touched=True)
        smod = clipped[0]  # smod band (e.g., 1=RES, 2=NRES, 3=Open Space)
        profile = src.profile
        profile.update({
            'height': smod.shape[0],
            'width': smod.shape[1],
            'transform': transform
        })
    except ValueError as e:
        print(f"Error: {e}")
        print("GeoTIFF Bounds:", src.bounds)
        print("Geometry Bounds:", gdf.geometry.bounds)
        raise

# Verify smod values
print("Unique smod values:", np.unique(smod))

Unique smod values: [  0   1   2 255]


In [17]:
res_mask = (smod == 1).astype(np.uint8)  # Residential
nres_mask = (smod == 2).astype(np.uint8)  # Non-residential
res_nres_mask = np.isin(smod, [1, 2]).astype(np.float32)  # RES+NRES for fractional
nres_only_mask = nres_mask.astype(np.uint8)  # Non-residential

# Save RES+NRES mask
with rasterio.open('res_nres_islamabad.tif', 'w', **profile) as dst:
    dst.write(res_nres_mask, 1)

In [40]:
import rasterio
from rasterio.warp import transform_bounds

# Define the base path to R10m folder
r10path = 'D://Ass//ANN//Project//S2A_MSIL2A_20181227T055231_N0500_R048_T43SCT_20230726T094312.SAFE//GRANULE//L2A_T43SCT_A018349_20181227T055233//IMG_DATA//R10m'

# Paths to .jp2 files
jp2_paths = {
    'B02': f'{r10path}//T43SCT_20181227T055231_B02_10m.jp2',
    'B03': f'{r10path}//T43SCT_20181227T055231_B03_10m.jp2',
    'B04': f'{r10path}//T43SCT_20181227T055231_B04_10m.jp2',
    'B08': f'{r10path}//T43SCT_20181227T055231_B08_10m.jp2'
}

# Inspect each band
for band, path in jp2_paths.items():
    with rasterio.open(path) as src:
        print(f"\nBand: {band}")
        print("CRS:", src.crs)  # Likely EPSG:32643 (UTM Zone 43N)
        print("Bounds:", src.bounds)  # In meters
        print("Shape:", src.shape)  # Height, width
        print("Bands:", src.count)  # Should be 1
        bounds_wgs84 = transform_bounds(src.crs, 'EPSG:4326', *src.bounds)
        print("Bounds in WGS84 (degrees):", bounds_wgs84)


Band: B02
CRS: EPSG:32643
Bounds: BoundingBox(left=300000.0, bottom=3690240.0, right=409800.0, top=3800040.0)
Shape: (10980, 10980)
Bands: 1
Bounds in WGS84 (degrees): (72.82621993951749, 33.33276710714379, 74.03062458149928, 34.33773660758858)

Band: B03
CRS: EPSG:32643
Bounds: BoundingBox(left=300000.0, bottom=3690240.0, right=409800.0, top=3800040.0)
Shape: (10980, 10980)
Bands: 1
Bounds in WGS84 (degrees): (72.82621993951749, 33.33276710714379, 74.03062458149928, 34.33773660758858)

Band: B04
CRS: EPSG:32643
Bounds: BoundingBox(left=300000.0, bottom=3690240.0, right=409800.0, top=3800040.0)
Shape: (10980, 10980)
Bands: 1
Bounds in WGS84 (degrees): (72.82621993951749, 33.33276710714379, 74.03062458149928, 34.33773660758858)

Band: B08
CRS: EPSG:32643
Bounds: BoundingBox(left=300000.0, bottom=3690240.0, right=409800.0, top=3800040.0)
Shape: (10980, 10980)
Bands: 1
Bounds in WGS84 (degrees): (72.82621993951749, 33.33276710714379, 74.03062458149928, 34.33773660758858)


In [43]:

from rasterio.warp import calculate_default_transform, reproject, Resampling

dst_crs = 'ESRI:54009'
reprojected_paths = {}
for band, jp2_path in jp2_paths.items():
    with rasterio.open(jp2_path) as src:
        data = src.read()
        profile = src.profile

        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds
        )
        profile.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height,
            'driver': 'GTiff'
        })

        reprojected = np.zeros((data.shape[0], height, width), dtype=data.dtype)
        reproject(
            source=data,
            destination=reprojected,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest
        )

        output_path = f's2_islamabad_{band}.tif'
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(reprojected)
        reprojected_paths[band] = output_path
        

In [44]:
b2 = rasterio.open(reprojected_paths['B02']).read(1)
b3 = rasterio.open(reprojected_paths['B03']).read(1)
b4 = rasterio.open(reprojected_paths['B04']).read(1)
b8 = rasterio.open(reprojected_paths['B08']).read(1)
stacked = np.stack([b2, b3, b4, b8], axis=0)

profile.update({'count': 4})
with rasterio.open('s2_islamabad_reprojected.tif', 'w', **profile) as dst:
    dst.write(stacked)

In [46]:
# Apply cloud mask using MSK_CLDPRB_20m
cld_path = r'D:\Ass\ANN\Project\S2A_MSIL2A_20181227T055231_N0500_R048_T43SCT_20230726T094312.SAFE\GRANULE\L2A_T43SCT_A018349_20181227T055233\QI_DATA\MSK_CLDPRB_20m.jp2'

with rasterio.open(cld_path) as src:
    cld = src.read(1)  # Cloud probability (0-100)
    cld_profile = src.profile

    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds
    )
    cld_profile.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })
    cld_reprojected = np.zeros((1, height, width), dtype=cld.dtype)
    reproject(
        source=cld[np.newaxis, :, :],
        destination=cld_reprojected,
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=transform,
        dst_crs=dst_crs,
        resampling=Resampling.nearest
    )
    cld_reprojected = cld_reprojected[0]

# Cloud mask: pixels with cloud probability < 20% are valid
cloud_mask = (cld_reprojected < 20).astype(np.uint8)  # Adjust threshold if needed


In [47]:

bbox = [73.0, 33.6, 73.2, 33.8]
geom = box(*bbox)
gdf = gpd.GeoDataFrame({'geometry': [geom]}, crs='EPSG:4326').to_crs('ESRI:54009')


In [48]:

with rasterio.open('s2_islamabad_reprojected.tif') as src:
    clipped, transform = mask(src, gdf.geometry, crop=True, all_touched=True)
    clipped_cloud_mask, _ = mask(rasterio.open('s2_islamabad_reprojected.tif'), gdf.geometry, crop=True, all_touched=True)
    clipped_cloud_mask = (clipped_cloud_mask[0] < 20).astype(np.uint8)
    clipped = clipped * clipped_cloud_mask[np.newaxis, :, :]
    profile.update({
        'height': clipped.shape[1],
        'width': clipped.shape[2],
        'transform': transform
    })

with rasterio.open('s2_islamabad.tif', 'w', **profile) as dst:
    dst.write(clipped)

In [5]:
from osgeo import gdal
import geopandas as gpd
from shapely.geometry import box
import rasterio
import numpy as np
import os

# Verify GDAL version
print("GDAL Version:", gdal.__version__)

# Enable GDAL exceptions
gdal.UseExceptions()

# Paths
ghs_path = r'GHS BUILT C\GHS_BUILT_C_FUN_E2018_GLOBE_R2023A_54009_10_V1_0_R5_C25.tif'
temp_clipped_path = r'D:\Ass\ANN\Project\clipped_ghs_temp.tif'
output_path = r'D:\Ass\ANN\Project\res_nres_islamabad.tif'

# Define study area with buffer (in EPSG:4326)
bbox = [72.9, 33.5, 73.3, 33.9]  # [minx, miny, maxx, maxy]
geom = box(*bbox)
gdf = gpd.GeoDataFrame({'geometry': [geom]}, crs='EPSG:4326')

# Step 1: Clip GHS-BUILT-C to small chunk in EPSG:4326
clip_options = gdal.WarpOptions(
    format='GTiff',
    outputBounds=bbox,
    dstSRS='EPSG:4326',
    creationOptions=['COMPRESS=LZW', 'TILED=YES'],
    resampleAlg=gdal.GRA_NearestNeighbour,
    multithread=True
)
gdal.Warp(temp_clipped_path, ghs_path, options=clip_options)

# Step 2: Reproject and clip to exact study area in ESRI:54009
study_bbox = [73.0, 33.6, 73.2, 33.8]  # Exact study area
study_geom = box(*study_bbox)
study_gdf = gpd.GeoDataFrame({'geometry': [study_geom]}, crs='EPSG:4326')
reprojected_bounds = study_gdf.to_crs('ESRI:54009').bounds.values[0]

warp_options = gdal.WarpOptions(
    format='GTiff',
    dstSRS='ESRI:54009',
    outputBounds=reprojected_bounds,
    xRes=10, yRes=10,  # Match Sentinel-2 10m resolution
    creationOptions=['COMPRESS=LZW', 'TILED=YES'],
    resampleAlg=gdal.GRA_NearestNeighbour,
    multithread=True
)
gdal.Warp(output_path, temp_clipped_path, options=warp_options)

# Clean up temporary file
if os.path.exists(temp_clipped_path):
    os.remove(temp_clipped_path)

print(f"Saved: {output_path}")

# Verify unique smod values and alignment
with rasterio.open(output_path) as src:
    smod = src.read(1)
    ghs_profile = src.profile
    print("GHS Unique smod values:", np.unique(smod))
    print("GHS Bounds:", src.bounds)
    print("GHS CRS:", src.crs)
    print("GHS Resolution:", src.res)

# Compare with Sentinel-2
with rasterio.open('s2_islamabad.tif') as src:
    print("Sentinel-2 Bounds:", src.bounds)
    print("Sentinel-2 CRS:", src.crs)
    print("Sentinel-2 Resolution:", src.res)

GDAL Version: 3.10.2
Saved: D:\Ass\ANN\Project\res_nres_islamabad.tif
GHS Unique smod values: [  0   1   2 255]
GHS Bounds: BoundingBox(left=6522846.155058076, bottom=4062239.0747620966, right=6550206.155058076, top=4085299.0747620966)
GHS CRS: ESRI:54009
GHS Resolution: (10.0, 10.0)
Sentinel-2 Bounds: BoundingBox(left=6522839.608420816, bottom=4062229.5651748986, right=6550211.8613917595, top=4085301.868172848)
Sentinel-2 CRS: ESRI:54009
Sentinel-2 Resolution: (12.28557135141083, 12.28557135141083)


In [6]:

with rasterio.open('s2_islamabad.tif') as src:
    img = src.read([1, 2, 3, 4])  # B2, B3, B4, B8
    img_profile = src.profile
    
with rasterio.open('res_nres_islamabad.tif') as src:
    mask = src.read(1)
    

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import rasterio
import numpy as np
import os


In [1]:
# Define Double Convolution Block
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        return self.double_conv(x)



NameError: name 'nn' is not defined

In [ ]:
# Define U-Net Model
class UNet(nn.Module):
    def __init__(self, in_channels=4, out_channels=1, features=[64, 128, 256, 512]):
        super(UNet, self).__init__()
        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Encoder
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature
        
        # Bottleneck
        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        
        # Decoder
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))
        
        # Final convolution
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)
        
        # Activation for fractional (sigmoid) or multi-class (none)
        self.out_channels = out_channels
        if out_channels == 1:
            self.final_act = nn.Sigmoid()
        else:
            self.final_act = nn.Identity()
    
    def forward(self, x):
        skip_connections = []
        
        # Encoder
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)
        
        # Bottleneck
        x = self.bottleneck(x)
        
        # Decoder
        skip_connections = skip_connections[::-1]
        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip = skip_connections[idx//2]
            x = torch.cat((skip, x), dim=1)
            x = self.ups[idx+1](x)
        
        # Final convolution
        x = self.final_conv(x)
        x = self.final_act(x)
        return x
    

In [ ]:

# Load and preprocess data
def load_data(s2_path, ghs_path, patch_size=256):
    # Load Sentinel-2
    with rasterio.open(s2_path) as src:
        img = src.read([1, 2, 3, 4])  # B2, B3, B4, B8
        img = img.transpose(1, 2, 0)  # (H, W, C)
    
    # Load GHS-BUILT-C
    with rasterio.open(ghs_path) as src:
        smod = src.read(1)  # smod band
    
    # Normalize Sentinel-2 (0–1)
    img = img.astype(np.float32)
    img_min = img.min(axis=(0, 1), keepdims=True)
    img_max = img.max(axis=(0, 1), keepdims=True)
    img = (img - img_min) / (img_max - img_min + 1e-8)
    
    # Prepare fractional labels (RES+NRES: smod=1,2)
    fractional_mask = np.where((smod == 1) | (smod == 2), 1.0, 0.0)
    
    # Prepare multi-class labels (0=Open Space, 1=RES, 2=NRES)
    multi_mask = smod.copy()
    multi_mask[smod == 255] = 0  # Treat no-data as Open Space
    
    # Extract patches
    patches_img = []
    patches_fractional = []
    patches_multi = []
    
    for i in range(0, img.shape[0], patch_size):
        for j in range(0, img.shape[1], patch_size):
            img_patch = img[i:i+patch_size, j:j+patch_size]
            frac_patch = fractional_mask[i:i+patch_size, j:j+patch_size]
            multi_patch = multi_mask[i:i+patch_size, j:j+patch_size]
            
            if img_patch.shape[:2] == (patch_size, patch_size):
                patches_img.append(img_patch.transpose(2, 0, 1))  # (C, H, W)
                patches_fractional.append(frac_patch[np.newaxis, ...])  # (1, H, W)
                patches_multi.append(multi_patch)  # (H, W)
    
    return (np.array(patches_img), 
            np.array(patches_fractional), 
            np.array(patches_multi))

In [ ]:

# Training function
def train_model(model, data_loader, criterion, optimizer, num_epochs, device):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, targets in data_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(data_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')


In [ ]:
# Paths
s2_path = r'D:\Ass\ANN\Project\s2_islamabad.tif'
ghs_path = r'D:\Ass\ANN\Project\res_nres_islamabad.tif'

# Load data
X, y_fractional, y_multi = load_data(s2_path, ghs_path, patch_size=256)

In [ ]:



# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y_fractional = torch.tensor(y_fractional, dtype=torch.float32)
y_multi = torch.tensor(y_multi, dtype=torch.long)

# Create datasets
dataset_fractional = TensorDataset(X, y_fractional)
dataset_multi = TensorDataset(X, y_multi)

# Create data loaders
batch_size = 4  # Small for CPU
loader_fractional = DataLoader(dataset_fractional, batch_size=batch_size, shuffle=True)
loader_multi = DataLoader(dataset_multi, batch_size=batch_size, shuffle=True)

# Device (CPU)
device = torch.device('cpu')

# Train Fractional U-Net
fractional_model = UNet(in_channels=4, out_channels=1).to(device)
criterion_fractional = nn.SmoothL1Loss()  # Huber loss equivalent
optimizer_fractional = optim.Adam(fractional_model.parameters(), lr=0.001)

print("Training Fractional U-Net")
train_model(fractional_model, loader_fractional, criterion_fractional, 
            optimizer_fractional, num_epochs=10, device=device)

# Save model
torch.save(fractional_model.state_dict(), 'fractional_unet.pt')

# Train Multi-Class U-Net
multi_model = UNet(in_channels=4, out_channels=3).to(device)
criterion_multi = nn.CrossEntropyLoss()
optimizer_multi = optim.Adam(multi_model.parameters(), lr=0.001)

print("Training Multi-Class U-Net")
train_model(multi_model, loader_multi, criterion_multi, 
            optimizer_multi, num_epochs=10, device=device)

# Save model
torch.save(multi_model.state_dict(), 'multi_unet.pt')

# Evaluation (example for fractional model)
def evaluate_model(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item() * inputs.size(0)
    return total_loss / len(data_loader.dataset)

# Evaluate Fractional U-Net
val_loss = evaluate_model(fractional_model, loader_fractional, criterion_fractional, device)
print(f'Fractional U-Net Validation Loss: {val_loss:.4f}')

# Evaluate Multi-Class U-Net
val_loss = evaluate_model(multi_model, loader_multi, criterion_multi, device)
print(f'Multi-Class U-Net Validation Loss: {val_loss:.4f}')

Training Fractional U-Net
Epoch 1/10, Loss: 0.1218
Epoch 2/10, Loss: 0.1149
Epoch 3/10, Loss: 0.1149
Epoch 4/10, Loss: 0.1169
Epoch 5/10, Loss: 0.1154
Epoch 6/10, Loss: 0.1147
Epoch 7/10, Loss: 0.1215
Epoch 8/10, Loss: 0.1193
Epoch 9/10, Loss: 0.1178
Epoch 10/10, Loss: 0.1188
Training Multi-Class U-Net
Epoch 1/10, Loss: 0.8825
Epoch 2/10, Loss: 0.7444
Epoch 3/10, Loss: 0.6887
Epoch 4/10, Loss: 0.7049
Epoch 5/10, Loss: 0.7027
Epoch 6/10, Loss: 0.6947
Epoch 7/10, Loss: 0.7138
Epoch 8/10, Loss: 0.6819
Epoch 9/10, Loss: 0.6887
Epoch 10/10, Loss: 0.6862
Fractional U-Net Validation Loss: 0.1616
Multi-Class U-Net Validation Loss: 0.6812
